In [4]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv('./data_with_anomalies.csv')
data.drop(["Anomaly_Score"],inplace=True,axis=1)
X = data.drop('Anomaly', axis=1)
Y = data[['Anomaly']]

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.75, test_size=0.25)

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')

D:\home\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
D:\home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9960927785748446

Generation 2 - Current best internal CV score: 0.9960927785748446

Generation 3 - Current best internal CV score: 0.9960927785748446

Generation 4 - Current best internal CV score: 0.9960927785748446

Generation 5 - Current best internal CV score: 0.9980439980870397

Best pipeline: RandomForestClassifier(LogisticRegression(MinMaxScaler(input_matrix), C=15.0, dual=False, penalty=l2), bootstrap=True, criterion=entropy, max_features=0.2, min_samples_leaf=8, min_samples_split=4, n_estimators=100)
0.9970674486803519


D:\home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MinMaxScaler
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('./data_with_anomalies.csv', sep=',', dtype=np.float64)
features = tpot_data.drop('Anomaly', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Anomaly'], random_state=42)

# Average CV score on the training set was: 0.9980439980870397
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    StackingEstimator(estimator=LogisticRegression(C=15.0, dual=False, penalty="l2")),
    RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.2, min_samples_leaf=8, min_samples_split=4, n_estimators=100)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [16]:
 from sklearn.metrics import classification_report
print(classification_report(testing_target, results))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       330
         1.0       1.00      1.00      1.00        11

    accuracy                           1.00       341
   macro avg       1.00      1.00      1.00       341
weighted avg       1.00      1.00      1.00       341

